In [ ]:
import datetime

import plotly.graph_objects as go
from plotly.subplots import make_subplots

from acquisition import fetch_candle_data
from processing.plot_utils import make_candlestick

from ta.trend import MACD
from ta.momentum import StochasticOscillator


candle_data = fetch_candle_data(ticker='NVDA', first_date=datetime.date(year=2019, month=1, day=1), last_date=datetime.date(year=2021, month=7, day=1))

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, row_heights=[60, 40])
fig.add_trace(make_candlestick(candle_data[30:200]), row=2, col=1)

threshold_buy = 0.02
threshold_sell = 0.01
target = candle_data.c.rolling(window=5).mean()
target = (candle_data.c - candle_data.c.shift(1)) / candle_data.c
target = target.dropna()
target = target.rolling(window=5, win_type='gaussian', center=True).mean(std=1)

target[(target < threshold_buy) & (target> -threshold_sell)] = 0
target[target > 0] = 1
target[target < 0] = -1

fig.add_scatter(y=target[30:200], x=candle_data.t[30:200], name='target', row=1, col=1)

macd = MACD(close=candle_data.c, window_fast=7, window_slow=13, window_sign=3).macd_signal()
stochastic = StochasticOscillator(high=candle_data.h, low=candle_data.l, close=candle_data.c, window=7).stoch_signal()/50 - 1

fig.add_scatter(y=macd[30:200], x=candle_data.t[30:200], name='MACD', row=1, col=1)
fig.add_scatter(y=stochastic[30:200], x=candle_data.t[30:200], name='stochastic', row=1, col=1)

fig.show()

In [ ]:
stochastic.corr(target)